## Multimodal Retrieval

### Deep Lake Rest API

In [16]:
import base64
from io import BytesIO
import json
import requests
import os
from PIL import Image

def retrieve_best_results(queries: list, org_id: str, dataset_name: str, k=4, number_of_images=3):
    url = f"https://beta.activeloop.dev/api/query/colpali/{org_id}/{dataset_name}"

    data = {
        "queries": queries,
        "k": k,
        "number_of_images": number_of_images,
    }

    headers = {
        "Authorization": f"Bearer {os.getenv('TOKEN')}",
        "Content-Type": "application/json",
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()


In [3]:

def save_images(value_returned: dict):
    for idx_question, img_list in enumerate(value_returned["images"]):
        for idx_img, img in enumerate(img_list):
            image_data = base64.b64decode(img)
            image = Image.open(BytesIO(image_data))
            image.save(f"question_{idx_question}_image_{idx_img}.jpg")


### Retrieve the best images and get the answer

In [6]:
import boto3
from botocore.exceptions import ClientError
from utils import get_image_message_structure


In [22]:
client = boto3.client("bedrock-runtime", region_name="us-east-1")
def get_bedrock_answer_with_images(question, image):
    # Start a conversation with the user message.
    messages = get_image_message_structure(image, question)
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
    try:
        # Send the message to the model, using a basic inference configuration.
        response = client.converse(
            modelId=model_id,
            messages=messages,
            inferenceConfig={"maxTokens": 2000, "temperature": 0},
            additionalModelRequestFields={"top_k": 250},
        )

        # Extract and print the response text.
        response_text = response["output"]["message"]["content"][0]["text"]
        return response_text

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)


In [20]:
org_id = "emanuelebeta"
dataset_name = "ingestion_ml_test2_colpali"
questions = ["describe the Gaussian distribution curve", "describe the family in the image"]

value_returned = retrieve_best_results(questions, org_id, dataset_name)
save_images(value_returned)

In [21]:
value_returned

{'description': 'Query successful.',
 'images': [['...']],
 'answers': ['No VLM used.', 'No VLM used.'],
 'indices': [[45, 98, 128, 558], [5, 459, 96, 137]]}

In [24]:

for idx, img_list in enumerate(value_returned["images"]):
    for img in img_list:
        byte_image = base64.b64decode(img)
        question_bedrock = questions[idx]

        answer = get_bedrock_answer_with_images(question_bedrock, byte_image)
        print("the answer is: ", answer)
        break


the answer is:  The image illustrates the likelihood function for a Gaussian distribution, shown by the red curve. The black points denote a data set of values {xn}, and the likelihood function given by (1.53) corresponds to the product of the blue values. The curve has a bell-shaped appearance, which is characteristic of the Gaussian or normal distribution probability density function. The likelihood function relates the observed data points to the unknown parameters (mean μ and variance σ^2) of the Gaussian distribution that the data is assumed to be drawn from.
the answer is:  The image shows a family of four sitting on a pebbly beach near the ocean. There are two adults - a man wearing a red shirt and a woman in a green top, along with two young boys. One boy is wearing a green t-shirt with a graphic design, while the other has a gray shirt. They all have protective glasses on, likely to safely view a solar eclipse mentioned in the caption below the photo. The family members are sm

## Deep Memory

In [25]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


def embedding_function(texts, model="text-embedding-3-large"):

    if isinstance(texts, str):
        texts = [texts]

    try:
        texts = [t.replace("\n", " ") for t in texts]
    except:
        pass
    return [
        data.embedding
        for data in openai.embeddings.create(input=texts, model=model).data
    ]


def retrieve_context_from_deeplake(vector_store_db, user_question, deep_memory):
    # deep memory inside the vectore store ==> deep_memory=True
    answer = vector_store_db.search(
        embedding_data=user_question,
        embedding_function=embedding_function,
        deep_memory=deep_memory,
        return_view=False,
        k=4,
    )
    return answer


### Load VectorStore

In [34]:
from deeplake.core.vectorstore import VectorStore
legal_dataset = "hub://activeloop/biomed_deep_memory_project_24"
vector_store = VectorStore(legal_dataset)

Deep Lake Dataset in hub://activeloop/biomed_deep_memory_project_24 already exists, loading from the storage


### Compare the answer with and without Deep Memory using Bedrock and Claude Sonnet

In [35]:
question = "How does the choice of T value affect the calculation of daily CFRs in the context of disease progression?"

deep_memory_chunks = retrieve_context_from_deeplake(
    vector_store, question, deep_memory=True
)
no_deep_memory_chunks = retrieve_context_from_deeplake(
    vector_store, question, deep_memory=False
)


In [36]:
deep_memory_chunks["text"]

['it should be realized that deaths at day X are averagely from cases at day X-T rather than day 75 X. Given a T value, a group of CFRs (daily CFRs) can be obtained from different X days. As 76 known that death number at day X should be less than case number at day X-T (if more than 77 day X-T, CFR would be greater than 100% which is illogical). Based on this point, the range 78 of T can be narrowed. More importantly, no matter what T value is assumed, even it is far 79 away from the true T value, the daily CFRs would converge towards (infinitely approach to 80 but never be over) the true CFR with time (X) increases. The following example will illustrate 81 this principle (Table 1) . Assuming CFR = 10%, T = 4 for a disease, the cases number was 82 from 100 to 10000 at day X (X=1 to 100), then the deaths number would be 10 (10, 20 and so 83 on) at day X+4 (5, 6 and so on). When calculating daily CFRs based on case and death 84 numbers with formula deaths (X) divided by cases (X-T), law 

In [37]:
no_deep_memory_chunks["text"]

['it should be realized that deaths at day X are averagely from cases at day X-T rather than day 75 X. Given a T value, a group of CFRs (daily CFRs) can be obtained from different X days. As 76 known that death number at day X should be less than case number at day X-T (if more than 77 day X-T, CFR would be greater than 100% which is illogical). Based on this point, the range 78 of T can be narrowed. More importantly, no matter what T value is assumed, even it is far 79 away from the true T value, the daily CFRs would converge towards (infinitely approach to 80 but never be over) the true CFR with time (X) increases. The following example will illustrate 81 this principle (Table 1) . Assuming CFR = 10%, T = 4 for a disease, the cases number was 82 from 100 to 10000 at day X (X=1 to 100), then the deaths number would be 10 (10, 20 and so 83 on) at day X+4 (5, 6 and so on). When calculating daily CFRs based on case and death 84 numbers with formula deaths (X) divided by cases (X-T), law 

In [32]:
from utils import get_text_message_structure

def get_bedrock_answer_with_text(question, chunks):
    # Start a conversation with the user message.
    messages = get_text_message_structure(chunks, question)
    try:
        # Send the message to the model, using a basic inference configuration.
        response = client.converse(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            messages=messages,
            inferenceConfig={"maxTokens": 2000, "temperature": 0},
            additionalModelRequestFields={"top_k": 250},
        )

        # Extract and print the response text.
        response_text = response["output"]["message"]["content"][0]["text"]
        return response_text

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

In [38]:
final_answer_deep_memory = get_bedrock_answer_with_text(question, deep_memory_chunks)
final_answer_no_deep_memory = get_bedrock_answer_with_text(
    question, no_deep_memory_chunks
)

According to the given text, the choice of T value (the average time period from case confirmation to death) affects the calculation of daily case fatality rates (CFRs) in the following ways:

1. For a given T value, a group of daily CFRs can be obtained from different days X by calculating deaths(X) / cases(X-T).

2. If the assumed T value is equal to the true T value, the calculated daily CFRs at different days X will constantly be equal to the true CFR.

3. If the assumed T value is greater than the true T value, the calculated daily CFRs will be greater than the true CFR initially, but will infinitely reduce and converge towards (but never exceed) the true CFR as time X increases.

4. If the assumed T value is less than the true T value, the calculated daily CFRs will be less than the true CFR initially, but will infinitely increase and converge towards (but never exceed) the true CFR as time X increases.

5. Importantly, no matter what T value is assumed, even if it is far away fr